In [48]:
#Check to see if gpu is available, should print 1
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available: 1


In [121]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, LeakyReLU

In [104]:
df = pd.read_csv("train.csv")
df.sample(10).head(10)

,cuneiform,lang
8970,𒊹,LTB
88315,𒂼𒌅𒌓𒁕𒀀𒀀𒌅𒌓𒁕𒁶𒆠𒁉𒂠𒄩𒈠𒄄𒄄,SUX
39304,𒀸𒊹𒌓𒋾𒋙𒃲,NEA
28387,𒉌𒄴𒄯,NEA
31766,𒊭𒌷𒂍𒃲𒈨𒌍,NEA
128580,𒀲𒉡𒊕𒋻,SUX
21849,𒈗𒂗𒅀𒆠𒄿𒈨𒂊𒌑𒋫𒈾𒁄𒋗,MPB
121459,𒊩𒅗,SUX
4947,𒊹𒃻𒀸𒌓𒊹𒄑𒊒𒁀𒋙𒇽𒋋𒌑𒌓𒅆𒊹𒀭𒌈𒀭𒅖𒋻,LTB
23907,𒌅𒌨𒊑𒍪𒈨𒊹𒌉𒇽𒅈𒉌𒈗𒁁𒉌𒅀,MPB


In [70]:
df.tail(10)

,cuneiform,lang,enc_lang
139411,𒄑𒀳𒋗𒌌,SUX,6
139412,𒄑𒀳𒋗𒉡𒌌,SUX,6
139413,𒄑𒀳𒄭𒆥,SUX,6
139414,𒄑𒀳𒀠𒍣𒊏,SUX,6
139415,𒄑𒀳𒋗,SUX,6
139416,𒄑𒀳𒋗𒌌,SUX,6
139417,𒄑𒀳𒋗𒉡𒌌,SUX,6
139418,𒄑𒀳𒄭𒆥,SUX,6
139419,𒄑𒀳𒀠𒍣𒊏,SUX,6
139420,𒄑𒀳𒋗,SUX,6


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139421 entries, 0 to 139420
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   cuneiform  139421 non-null  object
 1   lang       139421 non-null  object
 2   enc_lang   139421 non-null  int32 
dtypes: int32(1), object(2)
memory usage: 2.7+ MB


In [53]:
df.lang.value_counts()

lang
SUX    53673
NEA    32966
STB    17817
LTB    15947
NEB     9707
MPB     5508
OLB     3803
Name: count, dtype: int64

In [105]:
#Describe, 7 Different languages, 3223 samples are just the black circle, which is cuneiform for 3,600 or many or totality or world
#Cuneiform uses a 6 based numbering system, so 3,600 is 60*60 and not an arbitrary number or a dataset mistake

#No one hot encoding is needed, however, label is needed to be converted to integer value
#custom map to easily convert back at end
label_map = {'SUX': 0, 'NEA': 1, 'STB': 2, 'LTB': 3, 'NEB': 4, 'MPB': 5, 'OLB': 6}
df['enc_lang'] = df['lang'].map(label_map)
#ensure correct
df.sample(10).head(10)

,cuneiform,lang,enc_lang
89950,𒁯,SUX,0
96598,𒄘𒁺𒈝,SUX,0
75513,𒈪𒉭𒃻𒈪𒉭𒋢𒈪𒉭,STB,2
32216,𒊹𒈨𒌆𒈠𒃼𒋥𒊭𒊺𒅔𒉡,NEA,1
20908,𒅇𒁄𒋫𒋾𒊭𒉌𒌓𒋳𒈠,MPB,5
6391,𒊹,LTB,3
28640,𒃻𒈨𒌍𒊭𒇽𒄑𒇀𒈨𒌍,NEA,1
16050,𒇷𒅎𒉌𒋾𒆏,OLB,6
89074,𒌫,SUX,0
70938,𒅆𒊹𒊹,STB,2


In [106]:
#drop lang label
df.drop(columns=['lang'], inplace=True)
df.sample(10).head(10)

,cuneiform,enc_lang
126122,𒁕𒄃,0
114272,𒄀𒄥𒍥𒁕,0
123481,𒄷,0
6389,𒊹,3
96809,𒀭𒋗𒍣𒀭𒈾,0
34359,𒅅𒈠𒀀𒆠𒆷𒋙𒉡,1
90650,𒀭𒊩𒌆𒆠,0
81828,𒆕𒆕𒉿,2
91150,𒀭𒋗𒈤,0
76491,𒌓𒈠𒌓𒌓𒈠𒋫𒅆𒋡𒀀𒉿𒋼𒁉𒅁𒋾,2


In [ ]:
#need to split up cuneiform into vectors, then one hot encode those vectors for each symbol

In [107]:
#Stratify and split, 80% train, 20% test
X_train, X_test = train_test_split(df,test_size=.2,stratify=df['enc_lang'])

In [108]:
#Print sizes, only two labels, the cuneiform text and the language
print("X train shape -", X_train.shape)
print("X test shape -", X_test.shape)
print(X_train.head())
print(X_test.head())

X train shape - (111536, 2)
X test shape - (27885, 2)
                 cuneiform  enc_lang
130937                  𒍜𒋓         0
63008                    𒋙         4
1191                     𒋫         3
111747                 𒈬𒈬𒀀         0
5838    𒁕𒋻𒋝𒈬𒋥𒀀𒇉𒀭𒈹𒊹𒌑𒍑𒆠𒌑𒅎𒍇𒇻𒁕         3
                                             cuneiform  enc_lang
110996                                               𒄑         0
117984                                              𒄭𒂵         0
97126                                            𒋆𒋛𒀀𒅆𒂟         0
60322   𒀭𒊩𒌆𒆠𒃲𒀸𒊭𒀜𒈬𒅆𒀸𒈧𒈪𒄿𒍣𒍝𒈠𒅅𒁉𒄿𒋙𒊹𒂗𒇻𒀬𒅟𒀀𒈾𒆪𒈠𒋢𒌒𒉿𒄿𒅟𒇻𒍑𒈨𒄭𒋚𒋫𒅟𒇻𒆗𒅆𒅟         4
55995                                   𒂅𒌒𒍜𒅇𒄷𒌓𒊮𒁉𒀭𒈨𒌍𒃲𒈨𒌍         1


In [109]:
# Create your labels for each dataset, which would be the language
y_train = X_train['enc_lang'].copy()
X_train.drop('enc_lang', axis=1, inplace=True)

y_test = X_test['enc_lang'].copy()
X_test.drop('enc_lang', axis=1, inplace=True)